# R4openEO use case 1, machine learning UDF implementation

## Author btufail@eurac.edu
## Date 2022/09/28

### Import libraries and packages

In [2]:
from eo_utils import *

### Connect and Login

In [34]:
openeoHost = "https://openeo.eurac.edu"
conn = openeo.connect(openeoHost).authenticate_oidc(client_id="openEO_PKCE")

Authenticated using refresh token.


### Discover the available collections:

In [ ]:
conn.list_collections()

### Discover the available processes:

In [ ]:
conn.list_processes()

### Select the AOI
#### Use the rectangle selection tool to select the area of interest

In [38]:
center = [46.62, 10.95]
zoom = 15

eoMap = openeoMap(center,zoom)
eoMap.map

Map(center=[46.62, 10.95], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [39]:
bbox = eoMap.getBbox()
print("Coordinates selected from map:",'\n west',bbox[0],'\n east',bbox[2],'\n south',bbox[1],'\n north',bbox[3])

Coordinates selected from map: 
 west 10.904660224914553 
 east 10.965557098388674 
 south 46.62798526273379 
 north 46.639772524468164


In [40]:
spatial_extent  = {'west':bbox[0],'east':bbox[2],'south':bbox[1],'north':bbox[3]}

### Load data

In [41]:
conn = openeo.connect(openeoHost).authenticate_oidc(client_id="openEO_PKCE")

collection      = 'S2_L2A_ALPS'
temporal_extent = ["2020-07-01T00:00:00Z", "2020-09-01T00:00:00Z"]
bands = ["B02","B03","B04","B08"]

S2_data = conn.load_collection(collection,spatial_extent=spatial_extent,temporal_extent=temporal_extent, bands=bands)

Authenticated using refresh token.


In [43]:
S2_data_median = S2_data.reduce_dimension(reducer=median,dimension='DATE')

### Running reduce dimension UDF

In [44]:
udf_reducer = lambda data: data.run_udf(udf="https://raw.githubusercontent.com/Open-EO/r4openeo-usecases/main/uc1-ml-landcover/udfs/reduce_udf_chunk.R", runtime="R")

In [45]:
udf_result = S2_data_median.reduce_dimension(reducer=udf_reducer,dimension='bands')

### Running graph and saving results

In [46]:
result = udf_result.save_result(format='GTIFF')

In [47]:
job = conn.create_job(result,title="whatever1")
job_id = job.job_id
if job_id:
    print("Batch job created with id: ",job_id)
    job.start_job()
else:
    print("Error! Job ID is None")

Batch job created with id:  d514ba96-4732-456e-a00a-5a5af6c8bf0b


In [48]:
ST_S2_rgb_mask = job.get_results()
ST_S2_rgb_mask.download_files()

[PosixPath('/home/btufail@eurac.edu/Documents/SInCohMap/result.tiff'),
 PosixPath('/home/btufail@eurac.edu/Documents/SInCohMap/process.json')]